<a href="https://colab.research.google.com/github/SlowPoke228/Lessons/blob/main/L4(Tree).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from google.colab import drive
drive.mount('/content/drive')
# url db: https://www.kaggle.com/christianlillelund/csgo-round-winner-classification
# url lesson: http://slides.com/romvano/lab4
# https://colab.research.google.com/drive/1lob7X-2Cg2zDxobcOpeI7WjCsfSh8AmA#scrollTo=xe973iwk2POD
# https://colab.research.google.com/drive/1MsWimLckvkUunfErRUs0RDHbF6Em4-_z




names = ['time_left','ct_score','t_score','map','bomb_planted','ct_health','t_health','ct_armor','t_armor','ct_money','t_money','ct_helmets',
        't_helmets','ct_defuse_kits','ct_players_alive','t_players_alive','ct_weapon_ak47','t_weapon_ak47','ct_weapon_aug','t_weapon_aug','ct_weapon_awp'
        ,'t_weapon_awp','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','t_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_m4a4','t_weapon_m4a4','ct_weapon_mac10','t_weapon_mac10','ct_weapon_mag7'
         ,'t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','ct_weapon_mp9','t_weapon_mp9','ct_weapon_negev','t_weapon_negev','ct_weapon_nova',
        't_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver','ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20',
       't_weapon_scar20','ct_weapon_sg553','t_weapon_sg553','ct_weapon_ssg08','t_weapon_ssg08','ct_weapon_ump45','t_weapon_ump45','ct_weapon_xm1014',
       't_weapon_xm1014','ct_weapon_deagle','t_weapon_deagle','ct_weapon_fiveseven','t_weapon_fiveseven','ct_weapon_usps','t_weapon_usps','ct_weapon_p250','t_weapon_p250',
       'ct_weapon_p2000','t_weapon_p2000','ct_weapon_tec9','t_weapon_tec9','ct_grenade_hegrenade','t_grenade_hegrenade','ct_grenade_flashbang','t_grenade_flashbang',
       'ct_grenade_smokegrenade','t_grenade_smokegrenade','ct_grenade_incendiarygrenade','t_grenade_incendiarygrenade','ct_grenade_molotovgrenade','t_grenade_molotovgrenade',
       'ct_grenade_decoygrenade','t_grenade_decoygrenade','round_winner'
]

# Подключаем бд, удаляем первую стоку и указываем тайтлы
dataset = pd.read_csv("/content/drive/MyDrive/csgo_round_snapshots.csv", names=names, skiprows=1)
#удаляем данные с слишком высокой и слишком низкой корреляцией
dataset = dataset.drop(['map','bomb_planted','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_mac10','ct_weapon_mag7','t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','t_weapon_mp9',
        'ct_weapon_negev','t_weapon_negev','ct_weapon_nova','ct_weapon_xm1014','t_weapon_xm1014','t_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver',
        'ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20','t_weapon_scar20','ct_grenade_decoygrenade','t_grenade_decoygrenade','t_grenade_incendiarygrenade','ct_weapon_p2000','t_weapon_p2000',
        'ct_weapon_tec9','t_weapon_tec9','t_weapon_aug','ct_players_alive','t_players_alive','t_weapon_m4a4','t_weapon_ssg08','ct_grenade_molotovgrenade','ct_weapon_fiveseven','t_weapon_fiveseven'], 
        axis='columns')

dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_glock,ct_weapon_m4a4,t_weapon_mac10,ct_weapon_mp9,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_deagle,t_weapon_deagle,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_molotovgrenade,round_winner
0,175.00,0.0,0.0,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
2,96.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
3,76.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,T
122406,174.93,11.0,15.0,500.0,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,4.0,4.0,3.0,5.0,1.0,5.0,T
122408,94.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0,0.0,3.0,0.0,4.0,T


In [ ]:
# Создаем массивы с атрибутами и метками
X = dataset.iloc[:, :-1].values  
y_temp = dataset.iloc[:, len(dataset.columns)-1].values
y=[]

In [ ]:

#y=pd.DataFrame
for i in range(0,len(y_temp)):
  if y_temp[i]=='T':
    y.append(1.0)
  else:
    y.append(0.0)


Проверка стабильности одиночного дерева

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=42,stratify=y)
regressor = DecisionTreeRegressor(random_state=42,max_depth=30).fit(X_train, y_train)
print('Точность без уменьшения признаков',regressor.score(X_train, y_train))
X_train, X_test, y_train, y_test = train_test_split(X[:, 30:], y, test_size=0.40,random_state=42,stratify=y)
regressor = DecisionTreeRegressor(random_state=42,max_depth=30).fit(X_train, y_train)
print('Точность без первых 31 признаков',regressor.score(X_train, y_train))
X_train, X_test, y_train, y_test = train_test_split(X[:, :-30], y, test_size=0.40,random_state=42,stratify=y)
regressor = DecisionTreeRegressor(random_state=42,max_depth=30).fit(X_train, y_train)
print('Точность без последних 31 признаков',regressor.score(X_train, y_train))

Точность без уменьшения признаков 0.9624983306684665
Точность без первых 31 признаков 0.5867158215416024
Точность без последних 31 признаков 0.9371081696009912


In [ ]:
min_score=1
max_score=0
for i in range(0,10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=i,stratify=y)
  regressor = DecisionTreeRegressor(random_state=i,max_depth=10).fit(X_train, y_train)
  print(regressor.score(X_train, y_train))
  if max_score<regressor.score(X_train, y_train):
    max_score=regressor.score(X_train, y_train)
  if min_score>regressor.score(X_train, y_train):
    min_score=regressor.score(X_train, y_train)
print('Разность результатов при различных сидах рандома:', max_score-min_score) #дисперсия итоговой оценки

0.4491847506920833
0.45036417676985774
0.44888085526143917
0.448832814183027
0.4481627785871415
0.4518217269380145
0.44707006363214796
0.45314797248294114
0.4547456295273259
0.45471373531322734
Разность результатов при различных сидах рандома: 0.007675565895177927


Отбор признаков случайным лесом

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=42,stratify=y)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
feature_imp = pd.Series(clf.feature_importances_,index=dataset.iloc[:, :-1].columns).sort_values(ascending=False)
feature_imp

ct_armor                        0.093780
t_armor                         0.092524
t_money                         0.064342
ct_money                        0.063184
t_helmets                       0.057280
time_left                       0.049567
ct_health                       0.039756
t_health                        0.039617
t_score                         0.037027
ct_score                        0.036922
ct_defuse_kits                  0.035909
ct_helmets                      0.033025
ct_grenade_flashbang            0.029053
ct_weapon_usps                  0.025167
t_weapon_ak47                   0.023987
t_grenade_flashbang             0.022271
t_weapon_glock                  0.021651
ct_weapon_m4a4                  0.019742
ct_grenade_smokegrenade         0.018954
t_grenade_smokegrenade          0.017743
t_weapon_sg553                  0.017466
t_grenade_molotovgrenade        0.016243
ct_grenade_hegrenade            0.015834
ct_weapon_awp                   0.015251
ct_grenade_incen

Отбор признаков логистической регрессией

In [ ]:
from sklearn.linear_model import LogisticRegression
import math 
clf = LogisticRegression(random_state=0,solver='liblinear',penalty='l1').fit(X, y)
feature_imp = pd.Series(abs(clf.coef_[0]),index=dataset.iloc[:, :-1].columns).sort_values(ascending=False)

feature_imp

ct_weapon_awp                   0.351851
ct_weapon_aug                   0.327193
t_weapon_awp                    0.292588
ct_weapon_sg553                 0.286334
ct_weapon_m4a4                  0.284153
t_weapon_usps                   0.273199
ct_weapon_ssg08                 0.270812
t_weapon_ump45                  0.267290
t_weapon_sg553                  0.244806
t_helmets                       0.239887
ct_weapon_ak47                  0.222779
ct_weapon_mp9                   0.207344
t_weapon_ak47                   0.206990
t_weapon_deagle                 0.169914
ct_weapon_ump45                 0.168890
t_weapon_p250                   0.142265
t_grenade_smokegrenade          0.133304
ct_grenade_incendiarygrenade    0.117166
t_weapon_glock                  0.108786
ct_weapon_deagle                0.096334
t_weapon_mac10                  0.094921
t_grenade_molotovgrenade        0.094176
t_grenade_flashbang             0.091586
ct_weapon_usps                  0.056874
ct_helmets      

Сравнение работы случайного леса с использованием кросс-валидации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=42)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(n_estimators=10,min_samples_split=3)
print(cross_val_score(rfc, X, y_temp, cv=5))
rfc.fit(X_train, y_train)
print(rfc.score(X_train, y_train))



In [ ]:
rfr=RandomForestRegressor()
res=(cross_val_score(rfr, X_train_scaled, y_train, cv=5))
print(res.max())
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))

0.5457211575585309
0.9300497038971501


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=42)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
recall_score_rand_forest_max = -1
score_chart=np.ndarray([10,9])
best_est=0
best_depth=0
prev_score=0
i=0

for est in range(10,101,10):
  j=0
  for depth in range(7,16):
    rfc = RandomForestClassifier(n_estimators=est, max_depth=depth).fit(X_train, y_train)
    predicted_rfc = rfc.predict(X_test)
    recall_score_rand_forest_find = recall_score(y_test, predicted_rfc)
    if recall_score_rand_forest_max+0.005 < recall_score_rand_forest_find :
      recall_score_rand_forest_max = recall_score_rand_forest_find
      print(est, '   ', depth, '    ', recall_score_rand_forest_find,'        IMPROVEMENT')  
      best_est=est
      best_depth=depth
    score_chart[i,j]=(recall_score_rand_forest_find-prev_score) 
    prev_score=recall_score_rand_forest_find
    j+=1
  i+=1
print('recall_score_rand_forest_max:', recall_score_rand_forest_max)
rfcb=RandomForestClassifier(n_estimators=best_est, max_depth=best_depth)
print(cross_val_score(rfcb, X, y_temp, cv=5))






10     7      0.7192559559680206         IMPROVEMENT
10     9      0.74372263066972         IMPROVEMENT
10     13      0.7539271222530232         IMPROVEMENT
10     15      0.7738941786187779         IMPROVEMENT
20     15      0.7804829054678398         IMPROVEMENT
60     15      0.7881161865734603         IMPROVEMENT
recall_score_rand_forest_max: 0.7881161865734603
[0.74438363 0.75577976 0.75872069 0.7497345  0.73997222]


In [ ]:
score_chart

array([[ 7.19255956e-01, -1.95251296e-02,  4.39918043e-02,
        -1.13293962e-02,  1.43827086e-02, -3.21401310e-04,
         7.47258045e-03,  2.16945884e-03,  1.77975975e-02],
       [-4.37105781e-02, -1.82796995e-02,  2.20159897e-02,
         5.74504841e-03,  1.36595557e-03,  1.37800812e-02,
         2.12928368e-03,  1.15302720e-02,  1.20123740e-02],
       [-5.28705154e-02, -2.89261179e-03,  6.22715038e-03,
         6.06644972e-03,  1.60700655e-03,  1.65521675e-02,
         9.68221446e-03,  1.80788237e-03,  1.02044916e-02],
       [-5.99815194e-02,  1.29765779e-02,  8.03503274e-04,
         6.42802619e-03,  7.07082881e-03,  1.38202563e-02,
         4.05769154e-03,  1.10481700e-02,  1.11686955e-02],
       [-5.78522357e-02, -7.15117914e-03,  1.38202563e-02,
         7.79398176e-03,  1.40613073e-03,  1.47041099e-02,
         8.59748503e-03,  1.10481700e-02,  7.55293078e-03],
       [-5.76513599e-02, -1.38202563e-02,  2.14133623e-02,
         2.41050982e-03,  1.15704471e-02,  2.571210

Сравнение градиентного бустинга и леса

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=42)
clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.7531860142145249
CPU times: user 33.7 s, sys: 65.6 ms, total: 33.7 s
Wall time: 34.2 s


In [ ]:
%%time
rfc = RandomForestClassifier(random_state=0).fit(X_train, y_train)
print(rfc.n_features_in_)
print(rfc.score(X_train, y_train))

40
0.9956294420390491
CPU times: user 1min 27s, sys: 116 ms, total: 1min 27s
Wall time: 1min 28s
